```yaml
titan: v1
service:
  image: tensorflow
  machine:
    cpu: 2
    memory: 1024MB
```

In [27]:
import pandas as pd
import json
import re
import tweepy as tw
from nltk.corpus import stopwords
from textblob import TextBlob

In [28]:
consumer_key= 'your consumer_key here'
consumer_secret= 'your consumer_secret here'
access_token= 'your access token here'
access_token_secret= 'your access token secret here'

In [29]:
auth = tw.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)
api = tw.API(auth, wait_on_rate_limit=True)

In [30]:
def remove_url(txt):
    return " ".join(re.sub("([^0-9A-Za-z \t])|(\w+:\/\/\S+)", "", txt).split())

In [31]:
def get_tweets(search_term, amount, lang, date):

  tweets = tw.Cursor(api.search,
                   q=search_term,
                   lang=lang,
                   since=date).items(amount)
  
  if not tweets:
    raise TypeError('Error: not enough tweets available for your request. Please change your query')
  
  return tweets

In [32]:
def show_result(mean_sentiment):
 
  if mean_sentiment > 0.5:
    print('%8.2f Very Positive Sentiment' % (mean_sentiment))
  elif mean_sentiment >= 0.25 and mean_sentiment <= 0.5:
    print('%8.2f Positive Sentiment' % (mean_sentiment))
  elif mean_sentiment >= 0.0 and mean_sentiment <= 0.25:
    print('%8.2f Slightly Positive Sentiment' % (mean_sentiment))
  elif mean_sentiment < 0.0 and mean_sentiment >= -0.25:
    print('%8.2f Slightly Negative Sentiment' % (mean_sentiment))
  elif mean_sentiment < -0.25 and mean_sentiment >= -0.5:
    print('%8.2f Negative Sentiment' % (mean_sentiment))
  elif mean_sentiment < -0.5: 
    print('%8.2f Very Negative Sentiment' % (mean_sentiment))

In [33]:
# Mock request object for local API testing
args = {
    'param': ['#bitcoin']
}
REQUEST = json.dumps({ 'args': args })

In [34]:
# POST /sentiment
status = 200
location = None
content_type = 'application/json'
lang = 'en'
since = '2020-01-01'
amount = 100

try:
    
    request = json.loads(REQUEST)
    args = request.get('args', {})
    search_term = args.get('param', args.get('text', None))
 
    tweets = get_tweets(search_term, amount, lang, since)
    
    # Remove URLs
    tweets_no_urls = [remove_url(tweet.text) for tweet in tweets]

    # Create textblob objects of the tweets
    sentiment_objects = [TextBlob(tweet) for tweet in tweets_no_urls]

    # Check polarity
    sentiment_values = [[tweet.sentiment.polarity, str(tweet)] for tweet in sentiment_objects]
    
    sentiment_df = pd.DataFrame(sentiment_values, columns=["polarity", "tweet"])
    
    mean_sentiment = float(sentiment_df.mean())

    show_result(mean_sentiment) 
    
    
except Exception as err:
    status = 500
    content_type = 'application/json'
    print(json.dumps({ 'error': 'Cannot process request due to an error: {}'.format(err)}))

    0.09 Slightly Positive Sentiment
